In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

raw = pd.read_csv('./DATA/데이터2303_2310.csv', encoding='euc-kr', converters={'코드':str})
raw.shape

(13773, 18)

In [2]:
raw.head(1)

,코드,해촉일자,해촉사유,스코어2303,2209차월,2210차월,2211차월,2212차월,2301차월,2302차월,2303차월,2304차월,2305차월,2306차월,2307차월,2308차월,2309차월,2310차월
0,0001870469,NaN,NaN,758.0,3,4,5,6,7,8,9,10,11,12,13,14,15,16


In [3]:
min(raw[~raw.해촉일자.isna()].해촉일자)

20221001.0

In [4]:
data = raw[(raw['2302차월']>7)&(raw['2302차월']<38)]
data = data[(data.해촉사유.isna() )| ((data.해촉사유=='일반')&(data.해촉일자>20230215))]
data = data[['코드','해촉일자','해촉사유']]
data = data.set_index('코드')
data.head(1)

,해촉일자,해촉사유
코드,,
0001870469,NaN,NaN


In [5]:
data.shape

(5440, 2)

In [6]:
rank = pd.read_excel('../result/rank_2302_6M.xlsx', converters={'코드':str},
                    usecols = ['코드','proba','파일럿대상'])
rank = rank[['코드','proba','파일럿대상']]
rank = rank.set_index('코드')

In [7]:
sum(rank.파일럿대상)

437

In [8]:
df = data.join(rank)
# df[df.proba.isna()]
df = df[~df.proba.isna()]
# df.shape

# 등급별 이탈률
- 이탈 확률(`proba`)을 등급 갯수(`num_of_grade`)만큼 나누어 구간별 실제 이탈률 확인

In [10]:
min(df[~df.해촉일자.isna()].해촉일자)

20230301.0

In [40]:
bins = [0]
num_of_grade = 5
for i in range(num_cat-1):
    bins += [df.proba.quantile((i+1)/num_of_grade)]
bins += [1]
bins

[0,
 0.015072339658145788,
 0.05038449934624961,
 0.17589407324403084,
 0.5523355293188983,
 1]

In [41]:
labels = list(range(1, len(bins)))
df['grade'] = pd.cut(df['proba'], bins=bins, labels=labels, include_lowest=True)
df['y'] = list(map(lambda x: 1 if x=='일반' else 0, df.해촉사유))
df.head(1)

,해촉일자,해촉사유,proba,파일럿대상,grade,y
코드,,,,,,
0001870469,NaN,NaN,0.000366,False,1,0


In [42]:

round(df.proba.mean()*100,1)

24.0

In [43]:
grade_df = df.groupby(['grade']).agg(대상자수=('grade','size'),
                           이탈자수=('y','sum'),
                            평균점수=('proba','mean')
                                      ).reset_index()
# grade_df = grade_df.sort_values(by='grade', ascending=False)
grade_df['이탈률'] = grade_df.이탈자수/grade_df.대상자수
grade_df['평균점수'] = round(grade_df.평균점수*100,1)
grade_df

,grade,대상자수,이탈자수,평균점수,이탈률
0,1,1086,28,0.7,0.025783
1,2,1086,91,3.0,0.083794
2,3,1086,189,9.9,0.174033
3,4,1086,423,35.3,0.389503
4,5,1086,814,71.1,0.749540


# 상반기
- 상반기 파일럿 대상자의 6개월 이후 이탈률을 확인함

In [44]:
sum(df.파일럿대상)

422

In [45]:
grade_df = df.groupby(['grade']).agg(전체대상자=('grade','size'),
                         파일럿대상자=('파일럿대상', 'sum'),
                           이탈자수=('y','sum'),
                            이탈률=('y','mean'),
                            평균점수=('proba','mean')
                                      ).reset_index()

In [46]:
grade_df

,grade,전체대상자,파일럿대상자,이탈자수,이탈률,평균점수
0,1,1086,81,28,0.025783,0.006711
1,2,1086,86,91,0.083794,0.029870
2,3,1086,93,189,0.174033,0.098608
3,4,1086,89,423,0.389503,0.353136
4,5,1086,73,814,0.749540,0.711253


In [47]:
df2 = df[df.파일럿대상==True]
grade_df = df2.groupby(['grade']).agg(전체대상자=('grade','size'),
                         사전센싱대상자=('파일럿대상', 'sum'),
                           이탈자수=('y','sum'),
                            평균점수=('proba','mean')
                                      ).reset_index()
grade_df

,grade,전체대상자,사전센싱대상자,이탈자수,평균점수
0,1,81,81,2,0.007328
1,2,86,86,9,0.029686
2,3,93,93,13,0.104226
3,4,89,89,28,0.345559
4,5,73,73,45,0.700721


In [19]:
sum(df2.y)


97